In [ ]:
!pip install fastembed

In [2]:
from fastembed.embedding import TextEmbedding

# Loading model
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Obtain the embedding for the query
query = "I just discovered the course. Can I join now?"
embedding = list(model.embed([query]))[0]  # Obtain only the first value

# Review length and minimum value
print(f"Tamaño del embedding: {len(embedding)}")
print(f"Valor mínimo: {min(embedding)}")

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-22 15:37:03.322 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.


Tamaño del embedding: 512
Valor mínimo: -0.11726373885183883


In [3]:
import numpy as np

query = "I just discovered the course. Can I join now?"
doc = "Can I still join the course after the start date?"

# Obtain query and doc embeddings
query_vec = list(model.embed([query]))[0]
doc_vec = list(model.embed([doc]))[0]

# Convert to numpy arrays
query_vec = np.array(query_vec)
doc_vec = np.array(doc_vec)

# Calculate cosinus similarity
similarity = np.dot(query_vec, doc_vec)

print(f"Cosine similarity: {similarity:.3f}")


Cosine similarity: 0.901


In [4]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [5]:
# Document embeddings
doc_texts = [d["text"] for d in documents]
doc_embeddings = np.array(list(model.embed(doc_texts)))  # Matrix V

# Cosine similarities with all documents
similarities = doc_embeddings.dot(query_vec)

print(similarities)

# Get the index of the most similar document
most_similar_idx = np.argmax(similarities)

print(f"Most similar index: {most_similar_idx}")

[0.76296845 0.81823782 0.80853974 0.71330788 0.73044992]
Most similar index: 1


In [6]:
full_texts = [doc["question"] + " " + doc["text"] for doc in documents]

# Compute embeddings for the full_texts
full_embeddings = np.array(list(model.embed(full_texts)))

# Compute cosine similarities
similarities = full_embeddings.dot(query_vec)

# Print all similarities
for i, sim in enumerate(similarities):
    print(f"Doc {i}: similarity = {sim:.4f}")

# Get the index with the highest similarity
most_similar_idx = np.argmax(similarities)
print(f"Most similar index: {most_similar_idx}")

Doc 0: similarity = 0.8515
Doc 1: similarity = 0.8437
Doc 2: similarity = 0.8408
Doc 3: similarity = 0.7755
Doc 4: similarity = 0.8086
Most similar index: 0


In [7]:
# Because now we're including the "question" field in the embedding.
# Concatenating the question helps the model better understand the relevance, leading to a higher cosine similarity.

In [8]:
model = TextEmbedding(model_name="BAAI/bge-small-en")

# Test embedding
vec = list(model.embed(["test sentence"]))[0]
print(f"Vector length: {len(vec)}")  # Output: 384

Vector length: 384


In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp'}

In [5]:
from qdrant_client import QdrantClient

client = QdrantClient("http://localhost:6333")

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from qdrant_client.models import Distance, VectorParams, PointStruct

# Create collection
client.recreate_collection(
    collection_name="zoomcamp-homework",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

/tmp/ipykernel_27136/493805718.py:4: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [9]:
from fastembed import TextEmbedding
from tqdm import tqdm

model = TextEmbedding(model_name="BAAI/bge-small-en")
batch_size = 64
texts = [doc['question'] + ' ' + doc['text'] for doc in documents]

embeddings = []
for i in tqdm(range(0, len(texts), batch_size)):
    batch = texts[i:i+batch_size]
    embeddings.extend(model.embed(batch))


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [02:12<00:00, 22.16s/it]


In [12]:
from qdrant_client import models
import uuid

client.upload_points(
    collection_name="zoomcamp-homework",
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector=embeddings[i],
            payload={
                "text": documents[i]["text"],
                "question": documents[i]["question"],
                "course": documents[i]["course"],
            }
        )
        for i in range(len(documents))
    ]
)

In [15]:
query = "I just discovered the course. Can I join now?"
query_vector = list(model.embed([query]))[0]
search_result = client.search(
    collection_name="zoomcamp-homework",
    query_vector=query_vector,
    limit=1  # we only want the best result
)
print("Score:", search_result[0].score)

Score: 0.87031734


/tmp/ipykernel_27136/3988937341.py:3: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(
